In [3]:
import requests
from bs4 import BeautifulSoup
import zipfile
import io
import pandas as pd
import re

# URL of the page containing the ZIP file links
index_url = "https://bidarchive.relayscan.io/ethereum/mainnet/2024-07/index.html"

# Fetch the content of the page
response = requests.get(index_url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract all links that match the pattern
zip_urls = []
counter = 0
for link in soup.find_all('a', href=True):
    if counter > 2:
        break
    href = link['href']
    if re.match(r'\d{4}-\d{2}-\d{2}_all\.csv\.zip$', href):
        zip_urls.append(index_url.rsplit('/', 1)[0] + '/' + href)
        counter += 1


def download_and_extract_csv(zip_url):
    response = requests.get(zip_url)
    with zipfile.ZipFile(io.BytesIO(response.content)) as thezip:
        csv_files = [file for file in thezip.namelist() if file.endswith('.csv')]
        print(f"CSV files found: {csv_files}")  # Print the names of the CSV files
        frames = [pd.read_csv(thezip.open(file)) for file in csv_files]
        return pd.concat(frames, ignore_index=True)


# Combine all data into a single DataFrame
all_data = pd.concat([download_and_extract_csv(url) for url in zip_urls], ignore_index=True)


# Display the DataFrame
print(all_data.shape[0])




CSV files found: ['2024-07-01_all.csv']
CSV files found: ['2024-07-02_all.csv']
CSV files found: ['2024-07-03_all.csv']
48267191


In [4]:
from data_prep_utils import *
from plot_prep_utils import *

pd.set_option('display.float_format', lambda x: '%.2f' % x)
df_builder = get_builder_info_from_latest_TLDR_talk()

print(all_data.shape[0])

df_builder = get_builder_info_from_latest_TLDR_talk()
print(df_builder["builder_label"].unique())

48267191
['BTCS' 'Beelder' 'Blocknative' 'BuildAI' 'Builder Boi'
 'Building Blocks, Breaking Chains' 'Buildooooooooooooooooooooooor'
 'EigenPhi' 'Gambit Labs' 'I can haz block?'
 'If you build it, they will come.' "It's Free Real Estate" 'Manifold'
 'Titan' 'Ty For The Block' 'antbuilder' 'ashpool' 'beaverbuild'
 'bloXroute' 'bobTheBuilder' 'boba-builder' 'builder0x69' 'default key'
 'eden' 'eth-builder' 'f1b' 'finest artisanal blocks' 'flashbots'
 'ibuilder' 'imposter' 'jetbldr' 'lightspeedbuilder' 'lokibuilder'
 'manta-builder' 'mars' 'merkle Block' 'miao' 'nfactorial' 'panda'
 'payload' 'penguin' 'red ocean surfer' 'rsync' 's0e2t' 'smithbot'
 'smithbuilder' 'spiderbuilder' 'tbuilder' 'uwu.builders' 'wenmerge']


In [5]:
df_bid_with_merged_builder_label = pd.merge(all_data, df_builder, on='builder_pubkey', how='left')
print(df_bid_with_merged_builder_label.shape[0])


48267191


In [12]:
# Get the min and max block numbers
min_block_number = all_data['block_number'].min()
max_block_number = all_data['block_number'].max()
print(f'Min Block Number: {min_block_number}')
print(f'Max Block Number: {max_block_number}')

csv_filename = 'tx_block_'+ str(min_block_number) + '_to_' + str(max_block_number) + '.csv'

df_onchain_block = pd.read_csv(csv_filename)

# print("total block number: ")
# print(df_onchain_block['block_number'].nunique())

# 合并最新 builder label信息
df_onchain_block_with_merged_builder_label = pd.merge(df_onchain_block, df_builder, on='builder_pubkey', how='left', suffixes=('_old', ''))
print("total bids of the on chain blocks")
print(df_onchain_block.shape[0])

# 按照block number, value, relay进行groupby，计算 bid次数
df_onchain_unique_bids = df_onchain_block.groupby(['block_number', 'value', 'relay']).size().reset_index(name='bid_counts_by_key')
print("total unique bids of the on chain blocks")
print(df_onchain_unique_bids.shape[0])

# 按照block number, value, relay, builder_label进行groupby，计算 bid次数
df_onchain_unique_bids_by_builder = df_onchain_block.groupby(['block_number', 'value', 'relay', 'builder_label']).size().reset_index(name='bid_counts_by_builder')
print("total unique bids of the on chain blocks by builder")
print(df_onchain_unique_bids_by_builder.shape[0])


Min Block Number: 20207941
Max Block Number: 20229426
total bids of the on chain blocks
38291
total unique bids of the on chain blocks
36493
total unique bids of the on chain blocks by builder
36488


In [13]:
# 找出 df_onchain_unique_bids 和 df_onchain_unique_bids_by_builder 之间的差异行
# 首先，从 df_onchain_unique_bids_by_builder 中去掉 builder_label 列，以便比较
df_onchain_unique_bids_simplified = df_onchain_unique_bids_by_builder.drop(columns=['builder_label', 'bid_counts_by_builder']).copy()
df_onchain_unique_bids_simplified['key'] = 1  # 添加一个辅助列以便进行合并

# 添加一个辅助列到 df_onchain_unique_bids，以便进行合并
df_onchain_unique_bids['key'] = 1

# 使用 merge 合并两个数据帧
merged = pd.merge(df_onchain_unique_bids, df_onchain_unique_bids_simplified, on=['block_number', 'value', 'relay', 'key'], how='outer', indicator=True)

# 找出在 df_onchain_unique_bids 中存在但不在 df_onchain_unique_bids_simplified 中的行
difference_in_unique_bids = merged[merged['_merge'] == 'left_only']

# 找出在 df_onchain_unique_bids_simplified 中存在但不在 df_onchain_unique_bids 中的行
difference_in_unique_bids_by_builder = merged[merged['_merge'] == 'right_only']

# 打印差异行
print("Rows in df_onchain_unique_bids but not in df_onchain_unique_bids_by_builder:")
print(difference_in_unique_bids)

print("Rows in df_onchain_unique_bids_by_builder but not in df_onchain_unique_bids:")
print(difference_in_unique_bids_by_builder)


# 根据 difference_in_unique_bids 中的 block_number, value, relay 找出 df_onchain_block 中的对应数据行
matching_rows = df_onchain_block_with_merged_builder_label.merge(difference_in_unique_bids[['block_number', 'value', 'relay']], on=['block_number', 'value', 'relay'], how='inner')

# 打印找到的对应数据行
print("Matching rows in df_onchain_block for difference_in_unique_bids:")
print(matching_rows)

Rows in df_onchain_unique_bids but not in df_onchain_unique_bids_by_builder:
       block_number                  value                relay  \
626        20208309 1763680389588090112.00  bloxroute_regulated   
3134       20209740   68761545393998920.00           ultrasound   
17144      20217941  204953066076304576.00           ultrasound   
26317      20223399  617397212022392320.00           ultrasound   
34086      20228025  150153902827810816.00           ultrasound   

       bid_counts_by_key  key     _merge  
626                    1    1  left_only  
3134                   1    1  left_only  
17144                  1    1  left_only  
26317                  1    1  left_only  
34086                  1    1  left_only  
Rows in df_onchain_unique_bids_by_builder but not in df_onchain_unique_bids:
Empty DataFrame
Columns: [block_number, value, relay, bid_counts_by_key, key, _merge]
Index: []
Matching rows in df_onchain_block for difference_in_unique_bids:
        block_datetime  